#### 머신러닝 프로젝트
- 데이터셋 : used_car_data.csv
- 목표 : 다양한 모델로 데이터셋 분석 및 모델 생성.



In [1]:
# %pip install koreanize_matplotlib

In [2]:
## [1-1] 모듈로딩 
from _utils import tools as ts, RegressionModule as rm
from _utils.RegressionModule import RegressionModule as rm

import seaborn as sns
import pandas as pd # 데이터 분석 및 전처리
import numpy as np # 숫자처리
import matplotlib.pyplot as plt # 데이터 시각화
from sklearn.linear_model import LinearRegression ## ML 알고리즘
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
                            ## 성능평가 모듈
from sklearn.model_selection import train_test_split, KFold
                            ## 데이터셋 분리 관련 모듈
                            ## 학습/검증/테스트 
                                                    ## 교차검증\
from sklearn.preprocessing import PolynomialFeatures # 폴리. 컬럼추가
from sklearn.linear_model import Ridge, Lasso, ElasticNet            

from sklearn.utils.discovery import *
from sklearn.metrics import *   
from scipy import stats

import koreanize_matplotlib
                            

In [3]:
originDF = pd.DataFrame(pd.read_csv('../data/used_cars_data.csv',index_col='S.No.'))

In [4]:
carDF = originDF.copy()

컬럼설명<hr>
S.No.: 차량의 일련 번호 <br>
Name: 차량의 이름 (브랜드 이름과 모델 이름 포함)<br>
Location: 차량이 판매되거나 구매 가능한 위치 (도시)<br>
Year: 차량의 제조 연도<br>
Kilometers_driven: 이전 소유자가 차량을 운전한 총 거리 (KM 단위)<br>
Fuel_Type: 차량이 사용하는 연료 종류 (가솔린, 디젤, 전기, CNG, LPG)<br>
Transmission: 차량에 사용된 변속기 종류 (자동/수동)<br>
Owner: 차량의 소유 유형<br>
Mileage: 자동차 회사가 제공하는 표준 연비 (kmpl 또는 km/kg 단위)<br>
Engine: 차량 엔진의 배기량 (CC 단위)<br>
Power: 차량 엔진의 최대 출력 (bhp 단위)<br>
Seats: 차량의 좌석 수<br>
New_Price: 동일 모델의 새 차량 가격 (인도 루피 Lakhs 단위, 1 Lakh = 100,000 INR)<br>
Price: 중고차 가격 (인도 루피 Lakhs 단위, 1 Lakh = 100,000 INR<br>

In [5]:
aList = carDF[carDF['New_Price'].isna()]['Name'].unique().tolist()
with open('keyword', "w") as f:
      for item in aList:
        f.write(str(item) + "\n")  # Convert item to string and add newline

In [6]:
# 환율 (예시: 1 INR = 15 KRW)
exchange_rate = 16.75

# 중고차 가격을 한국 원화로 변환 후 백만 원 단위로 변환
carDF['Price'] = (carDF['Price'] * 100000 * exchange_rate) / 1000000

# # 결과 출력
print(carDF[['Price']])

          Price
S.No.          
0       2.93125
1      20.93750
2       7.53750
3      10.05000
4      29.71450
...         ...
7248        NaN
7249        NaN
7250        NaN
7251        NaN
7252        NaN

[7253 rows x 1 columns]


In [7]:
carDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7253 entries, 0 to 7252
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               7253 non-null   object 
 1   Location           7253 non-null   object 
 2   Year               7253 non-null   int64  
 3   Kilometers_Driven  7253 non-null   int64  
 4   Fuel_Type          7253 non-null   object 
 5   Transmission       7253 non-null   object 
 6   Owner_Type         7253 non-null   object 
 7   Mileage            7251 non-null   object 
 8   Engine             7207 non-null   object 
 9   Power              7207 non-null   object 
 10  Seats              7200 non-null   float64
 11  New_Price          1006 non-null   object 
 12  Price              6019 non-null   float64
dtypes: float64(2), int64(2), object(9)
memory usage: 793.3+ KB


In [8]:
## 각 컬럼별 고유값 갯수 확인
## 범주형, 연속형 데이터 확인
for i in carDF.columns:
    print(i, len(carDF[i].unique()))

Name 2041
Location 11
Year 23
Kilometers_Driven 3660
Fuel_Type 5
Transmission 2
Owner_Type 4
Mileage 451
Engine 151
Power 387
Seats 10
New_Price 626
Price 1374


In [9]:
carDF.isna().sum()
## 널값 상태 확인
## mileeage, Engine, Seats
## engine nan == power 도 없음 46 개이므로 폐기
## New_Price            6247가 null 폐기
## price 1234개는 추후 확인

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 46
Power                  46
Seats                  53
New_Price            6247
Price                1234
dtype: int64

In [10]:
carDF = carDF.drop(index = carDF[carDF['Engine'].isna()].index)
carDF.loc[carDF['Seats'].isna(),'Seats'] = [5,5,5,7,5,5,5]
carDF.loc[carDF['Mileage'].isna(), 'Mileage'] = [0,0]

In [11]:
## name 컬럼 앞의 회사명은 범주형으로 사용가능.

In [12]:
carDF.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
S.No.,,,,,,,,,,,,,
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,2.93125
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,20.93750
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,7.53750
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,10.05000
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,29.71450


In [13]:
## 수치지만 object인 컬럼 수정
# carDF['Mileage'] = carDF['Mileage'].apply(lambda x: x.split())
carDF['Mileage'] = carDF['Mileage'].apply(lambda x: str(x).split()[0] if isinstance(x, str) else x)
carDF["Engine"] = carDF["Engine"].apply(lambda x: str(x).split()[0])
carDF["Power"] = carDF["Power"].apply(lambda x: str(x).split()[0])


In [14]:
carDF.isnull().sum()

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 0
Engine                  0
Power                   0
Seats                   0
New_Price            6201
Price                1224
dtype: int64

In [15]:
carDF= carDF.drop(index=carDF[carDF['Power']=='null'].index)

In [16]:
#converting object data type to category data type
carDF["Fuel_Type"] = carDF["Fuel_Type"].astype("category")
carDF["Transmission"] = carDF["Transmission"].astype("category")
carDF["Owner_Type"] = carDF["Owner_Type"].astype("category")
#converting datatype  
carDF["Mileage"] = carDF["Mileage"].astype(float)
carDF["Power"] = carDF["Power"].astype(float)
carDF["Engine"]=carDF["Engine"].astype(float)

In [17]:
carDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7078 entries, 0 to 7252
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Name               7078 non-null   object  
 1   Location           7078 non-null   object  
 2   Year               7078 non-null   int64   
 3   Kilometers_Driven  7078 non-null   int64   
 4   Fuel_Type          7078 non-null   category
 5   Transmission       7078 non-null   category
 6   Owner_Type         7078 non-null   category
 7   Mileage            7078 non-null   float64 
 8   Engine             7078 non-null   float64 
 9   Power              7078 non-null   float64 
 10  Seats              7078 non-null   float64 
 11  New_Price          1006 non-null   object  
 12  Price              5876 non-null   float64 
dtypes: category(3), float64(5), int64(2), object(3)
memory usage: 629.5+ KB


In [18]:
## 차량 이름 앞의 브랜드명 추출하여 브랜드로, 모델명 추출하여 모델로
carDF['Brand'] = carDF['Name'].str.split(' ').str[0] #Separating Brand name from the Name
carDF['Model'] = carDF['Name'].str.split(' ').str[1] + carDF['Name'].str.split(' ').str[2]

carDF.Brand.unique()
## 'Isuzu', 'ISUZU' 같은걸로 보임.

array(['Maruti', 'Hyundai', 'Honda', 'Audi', 'Nissan', 'Toyota',
       'Volkswagen', 'Tata', 'Land', 'Mitsubishi', 'Renault',
       'Mercedes-Benz', 'BMW', 'Mahindra', 'Ford', 'Porsche', 'Datsun',
       'Jaguar', 'Volvo', 'Chevrolet', 'Skoda', 'Mini', 'Fiat', 'Jeep',
       'Ambassador', 'Isuzu', 'ISUZU', 'Force', 'Bentley', 'Lamborghini',
       'OpelCorsa'], dtype=object)

In [19]:
carDF['Brand'] = carDF['Brand'].str.replace('ISUZU','Isuzu' )
carDF.loc[carDF['Model'].isna(),'Model'] = 'Corsa'

In [20]:
carDF["Brand"] = carDF["Brand"].astype("category")
carDF["Model"] = carDF["Model"].astype("category")

In [21]:
## 제조년도와 현재년도를 이용해 연식 계산

carDF['Current_year']=2025
carDF['Ageofcar']=carDF['Current_year']-carDF['Year']
carDF.drop('Current_year',axis=1,inplace=True)

In [22]:
carDF.describe()

,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Price,Ageofcar
count,7078.000000,7.078000e+03,7078.000000,7078.000000,7078.000000,7078.000000,5876.000000,7078.000000
mean,2013.480503,5.825348e+04,18.268227,1620.717717,112.765214,5.284544,16.084464,11.519497
std,3.153077,8.521223e+04,4.369135,596.002717,53.493553,0.809451,18.837939,3.153077
min,1998.000000,1.710000e+02,0.000000,72.000000,34.200000,2.000000,0.737000,6.000000
25%,2012.000000,3.345875e+04,15.260000,1198.000000,75.000000,5.000000,5.891813,9.000000
50%,2014.000000,5.300000e+04,18.250000,1495.000000,94.000000,5.000000,9.631250,11.000000
75%,2016.000000,7.256600e+04,21.100000,1984.000000,138.100000,5.000000,16.770938,13.000000
max,2019.000000,6.500000e+06,33.540000,5998.000000,616.000000,10.000000,268.000000,27.000000


In [23]:
carDF.loc[carDF['Model']=='Prius2009-2016',:]

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Brand,Model,Ageofcar
S.No.,,,,,,,,,,,,,,,,
4904,Toyota Prius 2009-2016 Z4,Mumbai,2011,44000,Electric,Automatic,First,0.0,1798.0,73.0,5.0,NaN,21.35625,Toyota,Prius2009-2016,14


In [24]:
carDF.loc[carDF['Model']=='Prius2009-2016', 'Mileage'] = 23.91

In [25]:
carDF['Mileage'].groupby(carDF['Model']).median().sort_values().head(30)

C:\Users\kdt\AppData\Local\Temp\ipykernel_23756\4283817905.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  carDF['Mileage'].groupby(carDF['Model']).median().sort_values().head(30)


Model
C-ClassProgressive     0.00
Jazz2020               0.00
Superb3.6              0.00
EVerito                0.00
GallardoCoupe          6.40
Panamera2010           7.75
SL-ClassSL             8.10
ContinentalFlying      8.60
BoxsterS               8.60
M-ClassML              8.70
AccordV6               9.00
S60D4                  9.00
Superb2.8              9.30
XJ2.0L                 9.43
Pajero4X4              9.50
6Series                9.52
TT2.0                  9.90
CLS-Class2006-2010     9.90
E-Class230            10.00
370ZAT                10.00
E-ClassE400           10.00
E-Class250            10.00
CR-V2.4               10.10
GrandVitara           10.10
FlyingSpur            10.20
Z42009-2013           10.37
A62.8                 10.40
E-Class200            10.50
XJ5.0                 10.50
Scorpio2.6            10.50
Name: Mileage, dtype: float64

In [26]:
carDF.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Brand,Model,Ageofcar
S.No.,,,,,,,,,,,,,,,,
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.60,998.0,58.16,5.0,NaN,2.93125,Maruti,WagonR,15
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,NaN,20.93750,Hyundai,Creta1.6,10
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.20,1199.0,88.70,5.0,8.61 Lakh,7.53750,Honda,JazzV,14
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,NaN,10.05000,Maruti,ErtigaVDI,13
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,NaN,29.71450,Audi,A4New,12


price의 널값 1200개 처리하기.
- 5876/7078 = 0.8301780163888104
- 약 17%. test size를 0.17로 처리했다고 생각하고. 
- 예측모델을 만든 후 price 가 null인 것을 예측 모델에 넣어 데이터 만들기

- target - Price
- feature - 이름을 제외한 모든 컬럼.

In [27]:
pnullDF = carDF.loc[carDF[carDF['Price'].isna()].index, :]
newDF = carDF.loc[carDF[~carDF['Price'].isna()].index, :]

In [28]:
carDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7078 entries, 0 to 7252
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Name               7078 non-null   object  
 1   Location           7078 non-null   object  
 2   Year               7078 non-null   int64   
 3   Kilometers_Driven  7078 non-null   int64   
 4   Fuel_Type          7078 non-null   category
 5   Transmission       7078 non-null   category
 6   Owner_Type         7078 non-null   category
 7   Mileage            7078 non-null   float64 
 8   Engine             7078 non-null   float64 
 9   Power              7078 non-null   float64 
 10  Seats              7078 non-null   float64 
 11  New_Price          1006 non-null   object  
 12  Price              5876 non-null   float64 
 13  Brand              7078 non-null   category
 14  Model              7078 non-null   category
 15  Ageofcar           7078 non-null   int64   
dtypes: category

In [29]:
carDF.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Brand,Model,Ageofcar
S.No.,,,,,,,,,,,,,,,,
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.60,998.0,58.16,5.0,NaN,2.93125,Maruti,WagonR,15
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,NaN,20.93750,Hyundai,Creta1.6,10
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.20,1199.0,88.70,5.0,8.61 Lakh,7.53750,Honda,JazzV,14
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,NaN,10.05000,Maruti,ErtigaVDI,13
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,NaN,29.71450,Audi,A4New,12


In [30]:
Brand_dict = {num:i for i, num in enumerate(newDF['Brand'].unique())}
location_dict = {num:i for i, num in enumerate(newDF['Location'].unique())}
trans_dict = {num:i for i, num in enumerate(newDF['Transmission'].unique())}
fuel_dict = {num:i for i, num in enumerate(newDF['Fuel_Type'].unique())}
Model_dict = {num:i for i, num in enumerate(newDF['Model'].unique())}

In [31]:
# newDF['Location'] = newDF['Location'].replace(location_dict)
# newDF['Fuel_Type'] = newDF['Fuel_Type'].replace(fuel_dict)
# newDF['Transmission'] = newDF['Transmission'].replace(trans_dict)
# newDF['Brand'] = newDF['Brand'].replace(Brand_dict)
# newDF['Model'] = newDF['Model'].replace(Model_dict)

# 'Location' 컬럼에 대해 원핫 인코딩 수행

# 원본 DataFrame에 원핫 인코딩된 결과를 시리즈 형태로 추가
# 원핫 인코딩된 시리즈를 다시 원본 값으로 복원
# 1인 위치의 값을 찾아 해당하는 원본 값을 반환


In [32]:
## getdummy  컬럼별 적용

# for i in typeList:
#     source_encoded = pd.get_dummies(newDF[i], prefix=i)
#     newDF[i+'_Encoded'] = list(source_encoded.apply(lambda x: x.tolist(), axis=1))
#     newDF[i+'_Reversed'] = newDF[i+'_Encoded'].apply(lambda x: source_encoded.columns[x.index(1)] if 1 in x else None)



In [33]:
newDF.iloc[:,16:].head()

""
S.No.
0
1
2
3
4


In [34]:
from sklearn.ensemble import RandomForestRegressor

In [35]:
newDF['Car_size'] = np.where(newDF['Engine'] < 900, 1,   # 경차
                      np.where(newDF['Engine'] < 1500, 2,  # 소형차
                      np.where(newDF['Engine'] < 2000, 3,  # 중형차
                                               4)))

In [36]:
## 범주형 데이터들 라벨링하여 넘버링하기
## 로케이션, 브랜드, 모델은 순서 없음.
## 오너타입은 오디널.
##연식은 인코딩필요없음
from sklearn.preprocessing import  OrdinalEncoder


In [37]:
orEn = OrdinalEncoder()
newDF['Owner_Type'] = orEn.fit_transform(newDF[['Owner_Type']])


In [38]:
orEn.inverse_transform(newDF[['Owner_Type']])

array([['First'],
       ['First'],
       ['First'],
       ...,
       ['Second'],
       ['First'],
       ['First']], dtype=object)

In [39]:
newDF['Car_size'] = np.where(newDF['Engine'] < 1200, 1,   # 경차
                      np.where(newDF['Engine'] < 1500, 2,  # 소형차
                      np.where(newDF['Engine'] < 2000, 3,  # 중형차
                   4)))

In [40]:
newDF.columns


Index(['Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type',
       'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats',
       'New_Price', 'Price', 'Brand', 'Model', 'Ageofcar', 'Car_size'],
      dtype='object')

In [41]:
gDF = newDF.loc[:, ['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'Price', 'Ageofcar','Car_size'] ]

In [42]:
def pair_grid(data, *, hue=None, vars=None, x_vars=None, y_vars=None, 
              hue_order=None, palette=None, hue_kws=None, corner=False, 
              diag_sharey=True, height=2.5, aspect=1, layout_pad=0.5, 
              despine=True, dropna=False):
    g = sns.PairGrid(data=data, hue=hue, vars=vars, x_vars=x_vars, y_vars=y_vars, 
              hue_order=hue_order, palette=palette, hue_kws=hue_kws, corner=corner, 
              diag_sharey=diag_sharey, height=height, aspect=aspect, layout_pad=layout_pad, 
              despine=despine, dropna=dropna)
    g.map_upper(sns.kdeplot)
    g.map_diag( sns.kdeplot)
    # g.map_diag(lambda data, 
            #    **kwargs: 
            #     sns.histplot(data, kde=False, **{key: value for key, value in kwargs.items() if key != 'label'})
            #    or sns.kdeplot(data, **{key: value for key, value in kwargs.items() if key != 'label'}))

    # g.map_diag( sns.histplot, kde=True)
    g.map_lower(sns.scatterplot)    
    g.add_legend()

In [43]:
# # pair_grid(data=newDF[['Engine', 'Power', 'Price','Owner_Type']]
# #                ,hue='Owner_Type', palette='summer' )
# pair_grid(data=newDF[['Engine', 'Power', 'Price','Owner_Type']]
#                 )

In [44]:
# g = sns.pairplot(data=newDF[['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'Price', 'Ageofcar','Car_size']]
#                  ,corner=True, )
# labels =                    ["연도", "주행거리", "연비", "엔진", "출력", "좌석 수", "가격", "차량 연식", "차 크기"]

# # x축 라벨 변경 (마지막 행)
# for ax, label in zip(g.axes[-1, :], labels):
#     if ax is not None:
#         ax.set_xlabel(label, fontsize=24)

# # y축 라벨 변경 (첫 번째 열)
# for ax, label in zip(g.axes[:, 0], labels):
#     if ax is not None:
#         ax.set_ylabel(label, fontsize=24)
# plt.show()

In [45]:
# g = sns.pairplot(data=newDF[['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'Price', 'Ageofcar','Car_size']], corner=True)
# labels =                    ["연도", "주행거리", "연비", "엔진", "출력", "좌석 수", "가격", "차량 연식", "차 크기"]

# # x축 라벨 변경 (마지막 행)
# for ax, label in zip(g.axes[-1, :], labels):
#     if ax is not None:
#         ax.set_xlabel(label, fontsize=24)

# # y축 라벨 변경 (첫 번째 열)
# for ax, label in zip(g.axes[:, 0], labels):
#     if ax is not None:
#         ax.set_ylabel(label, fontsize=24)
# plt.show()

In [46]:
## 이상치 처리.
# 데이터프레임 예시
numeric_cols = ['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'Price', 'Ageofcar', 'Car_size']

# Z-스코어 계산
z_scores = np.abs(stats.zscore(newDF[numeric_cols]))

# 임계값 (보통 3을 사용)
threshold = 3

# 이상치가 아닌 데이터만 남기기
filtered_df = newDF[(z_scores < threshold).all(axis=1)]


In [47]:
newDF = newDF.drop(index=[340, 358, 2328, 1860, 4079, 5781, 5521])
newDF = newDF.drop(index=[2823, 3092, 4491])
newDF = newDF.drop(index=newDF[newDF['Mileage'] == 0].index)
newDF.shape

(5836, 17)

In [48]:
# newDF[(newDF['Price']>80)& (newDF['Year']<2010)]
newDF[newDF['Mileage'] <= 6]

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Brand,Model,Ageofcar,Car_size
S.No.,,,,,,,,,,,,,,,,,


In [49]:
# g = sns.pairplot(data=newDF[['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'Price', 'Ageofcar','Car_size']], corner=True, 
#                  plot_kws={'color': 'red'}, diag_kws={'color': 'red'} )
# labels = ["연도", "주행거리", "연비", "엔진", "출력", "좌석 수", "가격", "차량 연식", "차 크기"]

# # x축 라벨 변경 (마지막 행)
# for ax, label in zip(g.axes[-1, :], labels):
#     if ax is not None:
#         ax.set_xlabel(label, fontsize=24)

# # y축 라벨 변경 (첫 번째 열)
# for ax, label in zip(g.axes[:, 0], labels):
#     if ax is not None:
#         ax.set_ylabel(label, fontsize=24)
# plt.show()

In [50]:
import pandas as pd
import numpy as np

# 데이터 생성 예시
data = {'Value': [10, 12, 14, 15, 100, 200, 18, 16, 15, 30]}
df = pd.DataFrame(data)

# Z-Score 계산
mean = df['Value'].mean()
std_dev = df['Value'].std()

# Z-Score 계산
df[     'Z_Score'] = (df['Value'] - mean) / std_dev

# 이상치 정의: Z-Score 절댓값이 3보다 큰 값
df_no_outliers = df[df['Z_Score'].abs() <= 3]

print(df_no_outliers)


   Value   Z_Score
0     10 -0.537853
1     12 -0.505256
2     14 -0.472659
3     15 -0.456360
4    100  0.929019
5    200  2.558876
6     18 -0.407464
7     16 -0.440061
8     15 -0.456360
9     30 -0.211881


In [51]:
newDF.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Brand,Model,Ageofcar,Car_size
S.No.,,,,,,,,,,,,,,,,,
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,0.0,26.60,998.0,58.16,5.0,NaN,2.93125,Maruti,WagonR,15,1
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,0.0,19.67,1582.0,126.20,5.0,NaN,20.93750,Hyundai,Creta1.6,10,3
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,0.0,18.20,1199.0,88.70,5.0,8.61 Lakh,7.53750,Honda,JazzV,14,1
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,0.0,20.77,1248.0,88.76,7.0,NaN,10.05000,Maruti,ErtigaVDI,13,2
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,2.0,15.20,1968.0,140.80,5.0,NaN,29.71450,Audi,A4New,12,3


#### newDF 컬럼별 상관관계 분석

In [52]:
newDF['Kilometers_Driven'].sort_values()
# newDF['Kilometers_Driven'].describe()

S.No.
1361       171
5606       600
2054      1000
5941      1000
2646      1000
         ...  
1846    255000
29      262000
1975    282000
1528    299322
3649    300000
Name: Kilometers_Driven, Length: 5836, dtype: int64

In [53]:
# newDF = newDF.drop(index=2328)

In [54]:
# ts.plot_corr(newDF,featureDF.columns,'Price')

In [55]:
newDF['Kilometers_Driven'].describe()

count      5836.000000
mean      56698.982351
std       33509.796557
min         171.000000
25%       33493.250000
50%       52514.500000
75%       72202.500000
max      300000.000000
Name: Kilometers_Driven, dtype: float64

In [56]:
# for i in ['Year', 'Kilometers_Driven','Mileage', 'Engine', 'Power']:
#     sns.histplot(newDF[i], kde=True, bins=30)
#     plt.title(i)
#     plt.show()

In [57]:
#['Year', 'Kilometers_Driven','Mileage', 'Engine', 'Power']
# log       robust             standard     scipy.stats.boxcox         log
newDF = newDF.drop(columns=['Name', 'New_Price'])

In [58]:
# from sklearn.preprocessing import OneHotEncoder
# # OneHotEncoder 설정
# typeList = ['Location', 'Fuel_Type', 'Transmission', 'Brand', 'Model']

# encoder = OneHotEncoder(drop='first', sparse_output=False,handle_unknown='ignore')  # 첫 번째 카테고리 제거 (다중공선성 방지)

# # 범주형 데이터만 인코딩
# encoded_array = encoder.fit_transform(newDF[typeList])

# # 변환된 컬럼 이름 가져오기
# encoded_cols = encoder.get_feature_names_out(typeList)

# # 원-핫 인코딩된 데이터프레임 생성
# encoded_df = pd.DataFrame(encoded_array, columns=encoded_cols)
# encoded_df
# # 기존 숫자형 컬럼과 합치기
# # final_df = pd.concat([newDF.drop(columns=typeList), encoded_df], axis=1)

# # final_df.shape
# final_df = newDF.drop(columns=typeList).join(encoded_df)

In [59]:
targetSR = newDF['Price']
# featureDF = newDF[['Location_Encoded', 'Year', 'Kilometers_Driven', 'Fuel_Type_Encoded',
#        'Transmission_Encoded', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats',
#        'Brand_Encoded', 'Model_Encoded', 'Ageofcar']]

featureDF = newDF[['Mileage', 'Engine', 'Power','Ageofcar','Seats']]

In [60]:

X_train, X_test, y_train, y_test =ts.train_test_cut(featureDF,targetSR)
nModel = rm(X_train, X_test, y_train, y_test)

X_train => 2D (4377, 5) / X_test => 2D, (1459, 5)
y_train => 1D (4377,), / y_test => 1D, (1459,)


In [61]:
lmodel, adf = nModel.lr()
adf

,train_score,test_score,diff,train_loss,test_loss,coef,intercept
_utils.RegressionModule,0.698142,0.67796,0.020182,10.05816,10.199114,"[0.024098819057089254, 0.00380249354654661, 0....",12.070996


In [62]:
rfModel, rfDF = nModel.rf_regressor()

In [63]:
rfDF

,train_score,test_score,diff,train_loss,test_loss
_utils.RegressionModule,0.954925,0.902335,0.05259,3.886749,5.616643


In [64]:
rfModel.estimator_.__class__
rfModel.__class__.__name__

'RandomForestRegressor'

In [65]:
nModel.cv_regressor(rfModel)

,train_mean_r2,train_std_r2,test_mean_r2,test_std_r2,train_mean_rmse,train_std_rmse,test_mean_rmse,test_std_rmse
RandomForestRegressor,0.88661,0.023532,0.849687,0.045895,37.735135,6.889957,45.938183,9.176598
